In [1]:
import VariantAnnotation
import coverage
import pytest
import csv
from DAE import genomesDB

GA = genomesDB.get_genome()
gmDB = genomesDB.get_gene_models()

the twin 13772.s2 is not quite in the collection
set(['1995-09', '1998-11'])
birth month mismatch for twins from family 11665
the twin 12782.s2 is not quite in the collection


In [2]:
cov = coverage.Coverage()
cov.start()
pytest.main(['tests/VariantAnnotation_tests.py'])
cov.stop()
all_tests_missing_lines = set(cov.analysis2(VariantAnnotation)[3])

============================================================================================================================== test session starts ===============================================================================================================================
platform linux2 -- Python 2.7.12, pytest-3.0.3, py-1.4.31, pluggy-0.4.0
rootdir: /home/nikidimi/seqpipe/gpf/DAE, inifile: pytest.ini
plugins: cov-2.5.1, pylama-7.3.3

========================================================================================================================== no tests ran in 0.01 seconds ==========================================================================================================================


ERROR: file not found: tests/VariantAnnotation_tests.py
Coverage.py warning: No data was collected. (no-data-collected)


In [ ]:
filepath = "/home/nikidimi/seqpipe/data-dev-small/cccc/Chung2015CHD/Chung2015CHD.denovo-annot.txt"

In [ ]:
interesting_effects = []
with open("/home/nikidimi/seqpipe/data-dev-small/filenames_2", 'rb') as filenames:
    for line in filenames:
        filepath = "/home/nikidimi/seqpipe/data-dev-small/" + line.strip()
        print("Processing " + line.strip())
        with open(filepath, 'rb') as csvfile:
            reader = csv.DictReader(csvfile, delimiter='\t')
            for line in reader:
                try:
                    cov = coverage.Coverage()
                    cov.start()
                    _ = VariantAnnotation.annotate_variant(gmDB, GA, loc=line['location'], var=line['variant'])
                    cov.stop()
                    current_variant_missing = set(cov.analysis2(VariantAnnotation)[3])
                    diff_set = all_tests_missing_lines - current_variant_missing
                    if len(diff_set) > 0:
                        effect = {
                            'lines': diff_set,
                            'count': len(diff_set),
                            'data': line,
                            'var': line['variant'],
                            'loc': line['location']
                        }
                        interesting_effects.append(effect)
                except:
                    print("Ignoring exception")
                    pass

In [ ]:
sorted_effects = sorted(interesting_effects, key=lambda k: k['count'], reverse=True)

In [ ]:
sorted_effects

In [ ]:
all_lines = reduce(lambda acc, ef: acc | ef['lines'], sorted_effects, set())

In [ ]:
coverage = []
while len(all_lines) > 0:
    all_lines = all_lines - sorted_effects[0]['lines']
    coverage.append(sorted_effects[0])
    
    for effect in sorted_effects[1:]:
        effect['lines'] = effect['lines'] - sorted_effects[0]['lines']
        effect['count'] = len(effect['lines'])
    sorted_effects = sorted(sorted_effects[1:], key=lambda k: k['count'], reverse=True)

In [ ]:
len(sorted_effects)

In [ ]:
len(coverage)

In [ ]:

coverage[0]

In [ ]:
asserts_text = """
        self.assertEqual({0}.gene, {1})
        self.assertEqual({0}.transcript_id, {2})
        self.assertEqual({0}.strand, {3})
        self.assertEqual({0}.effect, {4})
        self.assertEqual({0}.prot_pos, {5})
        self.assertEqual({0}.prot_length, {6})
        self.assertEqual({0}.aa_change, {7})
"""

singe_res_text = """
    def {0}(self):
        [effect] = VariantAnnotation.annotate_variant(self.gmDB, self.GA,
                                                      loc="{1}",
                                                      var="{2}")
"""

multiple_res_text = """
    def {0}(self):
        effects = VariantAnnotation.annotate_variant(self.gmDB, self.GA,
                                                     loc="{1}",
                                                     var="{2}")
        self.assertEqual(len(effects), {3})
        effects_sorted = sorted(effects, key=lambda k: k.transcript_id)
"""

def add_quotes(string):
    if string is None:
        return string
    else:
        return "\"{}\"".format(string)

def generate_single_result_test(loc, var, effect, name):
    test_code = singe_res_text.format(name, loc, var)
    assert_code = asserts_text.format("effect",
                                      add_quotes(effect.gene), 
                                      add_quotes(effect.transcript_id),
                                      add_quotes(effect.strand), 
                                      add_quotes(effect.effect), 
                                      effect.prot_pos, effect.prot_length, 
                                      add_quotes(effect.aa_change))
    return test_code + assert_code

def generate_multi_line_result_test(loc, var, effects, name):
    effects_sorted = sorted(effects, key=lambda k: k.transcript_id)
    

    test_code = multiple_res_text.format(name, loc, var, len(effects_sorted))
    
    for i, effect in enumerate(effects_sorted):
        effects_elem = "effects_sorted[{}]".format(i)
        assert_code = asserts_text.format(effects_elem, 
                                          add_quotes(effect.gene), 
                                          add_quotes(effect.transcript_id),
                                          add_quotes(effect.strand), 
                                          add_quotes(effect.effect), 
                                          effect.prot_pos, effect.prot_length, 
                                          add_quotes(effect.aa_change))
        test_code += assert_code
    
    return test_code

def generate_test(loc, var):
    chrom = loc.split(":")[0]
    chrom_loc = loc.split(":")[1]
    name = "test_chr{0}_{1}_{2}_var".format(chrom, chrom_loc, var[:3])
    result = VariantAnnotation.annotate_variant(gmDB, GA, 
                                                loc=loc, 
                                                var=var)
    if len(result) == 1:
        return generate_single_result_test(loc, var, result[0], name)
    else:
        return generate_multi_line_result_test(loc, var, result, name)

In [ ]:
with open('autotests.py', 'w') as testsfile:
    for cov in coverage:
        testsfile.write(generate_test(cov['loc'], cov['var']))

In [ ]:
possible_effects = ["del(1)", "del(2)", "del(3)", 
                    "ins(A)", "ins(C)", "ins(G)", "ins(T)",
                    "sub(A->C)", "sub(A->G)", "sub(A->T)", 
                    "sub(C->A)", "sub(C->G)", "sub(C->T)", 
                    "sub(G->A)", "sub(G->C)", "sub(G->T)", 
                    "sub(T->A)", "sub(T->C)", "sub(T->G)"]

In [ ]:
def search_for_effects(i):
    interesting_effects = []
    for effect in possible_effects:
        cov = coverage.Coverage()
        cov.start()
        loc = "1:{}".format(i)
        _ = VariantAnnotation.annotate_variant(gmDB, GA, loc=loc, var=effect)
        cov.stop()
        current_variant_missing = set(cov.analysis2(VariantAnnotation)[3])
        diff_set = all_tests_missing_lines - current_variant_missing
        if len(diff_set) > 0:
            effect = {
                'lines': diff_set,
                'count': len(diff_set),
                'data': line,
                'var': line['variant'],
                'loc': line['location']
            }
            search_for_effects.q.put(effect)

def search_for_effects_init(q):
    search_for_effects.q = q

In [ ]:
len(interesting_effects)

In [ ]:
%%time
from multiprocessing import Process, Queue

q = Queue()
pool = Pool(None, search_for_effects_init, [q])
res = pool.map(search_for_effects, range(14600, 15000))

In [ ]:
while not q.empty():
    print(q.get())

In [ ]:
import gzip
with gzip.open("/home/nikidimi/seqpipe/data-dev-small/cccc/ExAC-0.1/ExAC-0.1.format-annot.txt.bgz") as f:
    reader = csv.DictReader(f, delimiter='\t')
    for line in reader:
        print(line)
        break

In [5]:
GA.getSequence("1", 897349, 897351)

'GGC'